In [1]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import concurrent.futures

In [2]:
foldertype="Acc_woGyro"
folder="Acc_wo"
folderg="Gyro"
path="C:\\Users\\nilu1\\Desktop\\Code\\AndroidPattern\\Data\\Train\\"+folder+"\\"
pathg="C:\\Users\\nilu1\\Desktop\\Code\\AndroidPattern\\Data\\Train\\"+folderg+"\\"
pathex=[]
temp = [f for f in listdir(path) if isfile(join(path, f))]
# temp.sort()
# files=["all"]
# files=["66b325","b56522","a99827"]
# files=["66b325","b56522"]
# files=["a99828","a99830"]
# files=["a99828","a99830","66b325","b56522"]
# files=["a29c12","a29c08","66b325","b56522"]
# files=["4b0c15"]
# files=["4b0c17","4b0c18"]
# files=["4b0c16","4b0c17","4b0c18"]
# files=["4b0c19"]
# files=["4b0c16","4b0c17","4b0c18","a29c08","66b325","b56522"]
# files=["4b0c16","4b0c17","4b0c18","a29c05"]
# files=["4b0c16","4b0c17","4b0c18","a29c14","a29c15"]
files=["4b0c16","4b0c17","4b0c18","a29c05","a29c14","a29c15"]
for i in range(len(temp)):
    st=temp[i]
    if(os.path.getsize(path+st)<4096):
        continue
    if(temp[i][:6] in files):
        pathex.append(temp[i])
#     if(files[0]=="all"):
#         pathex.append(temp[i])
# print(pathex)
print(len(temp))
print(len(pathex))

18726
9558


In [3]:
# folder_n_no.ofvalues_no.offiles_keycolno.
propath="C:\\Users\\nilu1\\Desktop\\Code\\AndroidPattern\\Data\\Processed\\"
def savefile():
    df.to_csv (propath+foldertype+"_"+str(n)+"_"+fcode+"_"+"&".join(files)+"_"+str(keypos)+".csv", index = False, header=True)

In [4]:
def acc4v(i):
    try:
        dff=pd.read_csv(path+pathex[i])
        mini=10000
        minkeyd=1000
        minkeyu=1000
        keydt=pathex[i].rfind("_")+4
        keyut=pathex[i].rfind("&")+1
        for j in dff.index:
            t=abs(dff['Time'][j] - int(pathex[i][keydt:keyut-1]))
            if(t < mini):
                mini=t
                minkeyd=j
        mini=10000
        for j in dff.index:
            t=abs(dff['Time'][j] - int(pathex[i][keyut:-4]))
            if(t < mini):
                mini=t
                minkeyu=j
    #     print(minkeyd)
    #     print(minkeyu)
        window=minkeyu-minkeyd
        left=int(nn/2)-window    
        dff=dff.head(minkeyu + left)
        dff=dff.tail(nn)
        dff.reset_index(inplace=True)
        dff=dff.drop(columns="index")
        dff=dff.drop(columns="Time")      
        return dff
    except:
        print("error : ",pathex[i])
#         traceback.print_exc()

In [5]:
#taking into account that error might occur while reading a file (putting try and except)
# n -> number of lines in csv file
# actual -> the actual key pressed
# minkeyd -> index of nearest key down time
# minkeyu -> index of nearest key up time
# keydt -> key down time from file name
# keyut -> key up time from file name
fcode="window"
nn=200
n=400+nn
keypos=1
actual=1
df=pd.read_csv(pathg+pathex[0])
for i in range(len(pathex)):
    try:
        dff=pd.read_csv(pathg+pathex[i])
        mini=10000
        minkeyd=1000
        minkeyu=1000
        keydt=pathex[i].rfind("_")+4
        keyut=pathex[i].rfind("&")+1
        for j in dff.index:
            t=abs(dff['Time'][j] - int(pathex[i][keydt:keyut-1]))
            if(t < mini):
                mini=t
                minkeyd=j
        mini=10000
        for j in dff.index:
            t=abs(dff['Time'][j] - int(pathex[i][keyut:-4]))
            if(t < mini):
                mini=t
                minkeyu=j
    #     print(minkeyd)
    #     print(minkeyu)
        window=minkeyu-minkeyd
        left=int((n-nn)/2)-window    
        dff=dff.head(minkeyu + left)
        dff=dff.tail(n-nn)
        dff.reset_index(inplace=True)
        dff=dff.drop(columns="index")
        dff=dff.drop(columns="Time")
        
        df1=acc4v(i)
        dff=dff.append(df1,ignore_index=True)        
        
        df1=dff.stack().swaplevel()
        df1.index=df1.index.map('{0[0]}_{0[1]}'.format) 
        dff=df1.to_frame().T
        ind=pathex[i].rfind("_")+3
        print(i,pathex[i])
        dff.insert(n*3, "key", int(pathex[i][ind]))
        keypos=n*3
        if(i==0):
            df=dff
        else:
            df=df.append(dff,ignore_index = True)
    except:
        print("error : ",pathex[i])
#         traceback.print_exec()
        continue

0 4b0c16-04-2020_09_17_43_AM01086&1185.csv
1 4b0c16-04-2020_09_17_45_AM1714&803.csv
2 4b0c16-04-2020_09_17_46_AM2707&788.csv
3 4b0c16-04-2020_09_17_47_AM3545&640.csv
4 4b0c16-04-2020_09_17_49_AM4596&714.csv
5 4b0c16-04-2020_09_17_50_AM5529&652.csv
6 4b0c16-04-2020_09_17_51_AM6500&601.csv
7 4b0c16-04-2020_09_17_52_AM7475&581.csv
8 4b0c16-04-2020_09_17_53_AM8469&595.csv
error :  4b0c16-04-2020_09_17_53_AM8469&595.csv
9 4b0c16-04-2020_09_17_54_AM9418&531.csv
error :  4b0c16-04-2020_09_17_54_AM9418&531.csv
10 4b0c16-04-2020_09_17_55_AM0431&554.csv
error :  4b0c16-04-2020_09_17_55_AM0431&554.csv
11 4b0c16-04-2020_09_17_57_AM2696&758.csv
12 4b0c16-04-2020_09_17_58_AM4682&759.csv
13 4b0c16-04-2020_09_18_00_AM6896&1007.csv
14 4b0c16-04-2020_09_18_01_AM8759&847.csv
15 4b0c16-04-2020_09_18_02_AM0595&715.csv
16 4b0c16-04-2020_09_18_04_AM3679&731.csv
17 4b0c16-04-2020_09_18_05_AM5671&739.csv
18 4b0c16-04-2020_09_18_06_AM7629&740.csv
19 4b0c16-04-2020_09_18_08_AM9735&828.csv
20 4b0c16-04-2020_09_18

186 4b0c16-04-2020_09_23_25_AM4339&437.csv
error :  4b0c16-04-2020_09_23_25_AM4339&437.csv
187 4b0c16-04-2020_09_23_26_AM5371&518.csv
error :  4b0c16-04-2020_09_23_26_AM5371&518.csv
188 4b0c16-04-2020_09_23_27_AM6257&373.csv
error :  4b0c16-04-2020_09_23_27_AM6257&373.csv
189 4b0c16-04-2020_09_23_28_AM7527&679.csv
190 4b0c16-04-2020_09_23_29_AM8506&670.csv
191 4b0c16-04-2020_09_23_30_AM9286&361.csv
error :  4b0c16-04-2020_09_23_30_AM9286&361.csv
192 4b0c16-04-2020_09_23_31_AM0242&357.csv
error :  4b0c16-04-2020_09_23_31_AM0242&357.csv
193 4b0c16-04-2020_09_23_32_AM2646&731.csv
194 4b0c16-04-2020_09_23_33_AM4365&439.csv
error :  4b0c16-04-2020_09_23_33_AM4365&439.csv
195 4b0c16-04-2020_09_23_34_AM6359&475.csv
error :  4b0c16-04-2020_09_23_34_AM6359&475.csv
196 4b0c16-04-2020_09_23_35_AM8568&689.csv
197 4b0c16-04-2020_09_23_37_AM0462&547.csv
198 4b0c16-04-2020_09_23_38_AM3864&955.csv
199 4b0c16-04-2020_09_23_39_AM5688&761.csv
200 4b0c16-04-2020_09_23_41_AM7578&683.csv
201 4b0c16-04-2020_

360 4b0c16-04-2020_09_27_51_AM7594&646.csv
361 4b0c16-04-2020_09_27_52_AM6492&607.csv
362 4b0c16-04-2020_09_27_53_AM5439&539.csv
363 4b0c16-04-2020_09_27_54_AM4443&516.csv
364 4b0c16-04-2020_09_27_55_AM3505&624.csv
365 4b0c16-04-2020_09_27_56_AM2411&508.csv
error :  4b0c16-04-2020_09_27_56_AM2411&508.csv
366 4b0c16-04-2020_09_27_57_AM1390&472.csv
error :  4b0c16-04-2020_09_27_57_AM1390&472.csv
367 4b0c16-04-2020_09_27_58_AM0388&515.csv
error :  4b0c16-04-2020_09_27_58_AM0388&515.csv
368 4b0c16-04-2020_09_30_57_AM0711&810.csv
369 4b0c16-04-2020_09_30_58_AM1608&705.csv
370 4b0c16-04-2020_09_30_59_AM2393&513.csv
error :  4b0c16-04-2020_09_30_59_AM2393&513.csv
371 4b0c16-04-2020_09_31_00_AM3312&423.csv
error :  4b0c16-04-2020_09_31_00_AM3312&423.csv
372 4b0c16-04-2020_09_31_02_AM4531&626.csv
373 4b0c16-04-2020_09_31_03_AM5292&444.csv
error :  4b0c16-04-2020_09_31_03_AM5292&444.csv
374 4b0c16-04-2020_09_31_04_AM6263&420.csv
error :  4b0c16-04-2020_09_31_04_AM6263&420.csv
375 4b0c16-04-2020_

533 4b0c16-04-2020_09_34_47_AM2679&814.csv
534 4b0c16-04-2020_09_34_49_AM0678&772.csv
535 4b0c16-04-2020_09_34_50_AM9643&755.csv
536 4b0c16-04-2020_09_34_51_AM7675&756.csv
537 4b0c16-04-2020_09_34_53_AM5625&739.csv
538 4b0c16-04-2020_09_34_54_AM3621&773.csv
539 4b0c16-04-2020_09_34_56_AM11058&1180.csv
540 4b0c16-04-2020_09_34_57_AM9670&740.csv
541 4b0c16-04-2020_09_34_58_AM8517&649.csv
542 4b0c16-04-2020_09_34_59_AM7517&611.csv
543 4b0c16-04-2020_09_35_01_AM6475&599.csv
544 4b0c16-04-2020_09_35_02_AM5355&464.csv
error :  4b0c16-04-2020_09_35_02_AM5355&464.csv
545 4b0c16-04-2020_09_35_03_AM4441&558.csv
546 4b0c16-04-2020_09_35_04_AM3352&473.csv
error :  4b0c16-04-2020_09_35_04_AM3352&473.csv
547 4b0c16-04-2020_09_35_05_AM2354&484.csv
error :  4b0c16-04-2020_09_35_05_AM2354&484.csv
548 4b0c16-04-2020_09_35_06_AM1395&493.csv
error :  4b0c16-04-2020_09_35_06_AM1395&493.csv
549 4b0c16-04-2020_09_35_07_AM0358&500.csv
error :  4b0c16-04-2020_09_35_07_AM0358&500.csv
550 4b0c16-04-2020_09_53_28

707 4b0c16-04-2020_09_57_11_AM2656&755.csv
708 4b0c16-04-2020_09_57_12_AM9617&727.csv
709 4b0c16-04-2020_09_57_14_AM6638&745.csv
710 4b0c16-04-2020_09_57_15_AM3665&797.csv
711 4b0c16-04-2020_09_57_16_AM0719&874.csv
712 4b0c16-04-2020_09_57_18_AM8558&702.csv
713 4b0c16-04-2020_09_57_19_AM6617&750.csv
714 4b0c16-04-2020_09_57_20_AM4631&750.csv
715 4b0c16-04-2020_09_57_22_AM2580&738.csv
716 4b0c16-04-2020_09_57_23_AM0707&805.csv
717 4b0c16-04-2020_09_57_24_AM9674&765.csv
718 4b0c16-04-2020_09_57_26_AM7658&792.csv
719 4b0c16-04-2020_09_57_27_AM5652&762.csv
720 4b0c16-04-2020_09_57_29_AM3631&821.csv
721 4b0c16-04-2020_09_57_30_AM1660&767.csv
722 4b0c16-04-2020_09_57_31_AM9644&747.csv
723 4b0c16-04-2020_09_57_32_AM8332&482.csv
error :  4b0c16-04-2020_09_57_32_AM8332&482.csv
724 4b0c16-04-2020_09_57_33_AM7407&524.csv
error :  4b0c16-04-2020_09_57_33_AM7407&524.csv
725 4b0c16-04-2020_09_57_35_AM6566&716.csv
726 4b0c16-04-2020_09_57_36_AM5448&571.csv
727 4b0c16-04-2020_09_57_37_AM4574&679.csv
7

...

8711 a29c15-05-2020_08_54_24_AM1733&865.csv
8712 a29c15-05-2020_08_54_25_AM8801&932.csv
8713 a29c15-05-2020_08_54_27_AM5641&727.csv
8714 a29c15-05-2020_08_54_28_AM2620&705.csv
8715 a29c15-05-2020_08_54_29_AM9799&908.csv
8716 a29c15-05-2020_08_54_31_AM6779&838.csv
8717 a29c15-05-2020_08_54_32_AM3559&629.csv
8718 a29c15-05-2020_08_54_33_AM0896&1002.csv
8719 a29c15-05-2020_08_54_35_AM8679&773.csv
8720 a29c15-05-2020_08_54_36_AM6910&985.csv
8721 a29c15-05-2020_08_54_38_AM4815&944.csv
8722 a29c15-05-2020_08_54_39_AM2723&814.csv
8723 a29c15-05-2020_08_54_41_AM0774&891.csv
8724 a29c15-05-2020_08_54_42_AM9780&909.csv
8725 a29c15-05-2020_08_54_44_AM7837&921.csv
8726 a29c15-05-2020_08_54_45_AM5610&712.csv
8727 a29c15-05-2020_08_54_46_AM3499&609.csv
8728 a29c15-05-2020_08_54_47_AM1617&714.csv
8729 a29c15-05-2020_08_54_49_AM9846&964.csv
8730 a29c15-05-2020_08_54_50_AM8542&630.csv
8731 a29c15-05-2020_08_54_51_AM7514&578.csv
8732 a29c15-05-2020_08_54_52_AM6523&668.csv
8733 a29c15-05-2020_08_54_53_AM

8895 a29c15-05-2020_08_59_02_AM5757&866.csv
8896 a29c15-05-2020_08_59_04_AM2689&764.csv
8897 a29c15-05-2020_08_59_05_AM9742&865.csv
8898 a29c15-05-2020_08_59_06_AM6781&830.csv
8899 a29c15-05-2020_08_59_08_AM3688&772.csv
8900 a29c15-05-2020_08_59_09_AM0784&875.csv
8901 a29c15-05-2020_08_59_11_AM8961&1029.csv
8902 a29c15-05-2020_08_59_13_AM61386&1489.csv
8903 a29c15-05-2020_08_59_14_AM4916&997.csv
8904 a29c15-05-2020_08_59_16_AM2767&848.csv
8905 a29c15-05-2020_08_59_17_AM0747&879.csv
8906 a29c15-05-2020_08_59_19_AM9761&846.csv
8907 a29c15-05-2020_08_59_20_AM7943&1027.csv
8908 a29c15-05-2020_08_59_22_AM5757&827.csv
8909 a29c15-05-2020_08_59_23_AM3733&809.csv
8910 a29c15-05-2020_08_59_25_AM1853&894.csv
8911 a29c15-05-2020_08_59_26_AM9732&844.csv
8912 a29c15-05-2020_08_59_27_AM8778&829.csv
8913 a29c15-05-2020_08_59_29_AM7778&866.csv
8914 a29c15-05-2020_08_59_30_AM6792&863.csv
8915 a29c15-05-2020_08_59_32_AM5854&886.csv
8916 a29c15-05-2020_08_59_33_AM4721&771.csv
8917 a29c15-05-2020_08_59_34

9078 a29c15-05-2020_09_04_39_AM2641&708.csv
9079 a29c15-05-2020_09_04_41_AM91092&1192.csv
9080 a29c15-05-2020_09_04_42_AM6708&776.csv
9081 a29c15-05-2020_09_04_44_AM3469&552.csv
9082 a29c15-05-2020_09_04_45_AM0726&818.csv
9083 a29c15-05-2020_09_04_46_AM8738&809.csv
9084 a29c15-05-2020_09_04_48_AM6820&907.csv
9085 a29c15-05-2020_09_04_49_AM4773&885.csv
9086 a29c15-05-2020_09_04_51_AM2616&691.csv
9087 a29c15-05-2020_09_04_52_AM0966&1102.csv
9088 a29c15-05-2020_09_04_54_AM9743&828.csv
9089 a29c15-05-2020_09_04_55_AM7824&958.csv
9090 a29c15-05-2020_09_04_56_AM5520&581.csv
9091 a29c15-05-2020_09_04_58_AM3644&725.csv
9092 a29c15-05-2020_09_04_59_AM1598&665.csv
9093 a29c15-05-2020_09_05_00_AM9729&830.csv
9094 a29c15-05-2020_09_05_01_AM8606&683.csv
9095 a29c15-05-2020_09_05_03_AM7514&587.csv
9096 a29c15-05-2020_09_05_04_AM6599&687.csv
9097 a29c15-05-2020_09_05_05_AM5520&584.csv
9098 a29c15-05-2020_09_05_06_AM4553&632.csv
9099 a29c15-05-2020_09_05_07_AM3622&714.csv
9100 a29c15-05-2020_09_05_08_

9240 a29c15-05-2020_09_09_19_AM2857&907.csv
9241 a29c15-05-2020_09_09_22_AM72332&2516.csv
9242 a29c15-05-2020_09_09_24_AM2636&705.csv
9243 a29c15-05-2020_09_09_25_AM8779&900.csv
9244 a29c15-05-2020_09_09_26_AM3742&792.csv
9245 a29c15-05-2020_09_09_28_AM8686&772.csv
9246 a29c15-05-2020_09_09_29_AM4747&818.csv
9247 a29c15-05-2020_09_09_31_AM91031&1106.csv
9248 a29c15-05-2020_09_09_32_AM4923&1018.csv
9249 a29c15-05-2020_09_09_34_AM0720&848.csv
9250 a29c15-05-2020_09_09_35_AM7812&961.csv
9251 a29c15-05-2020_09_09_37_AM3756&799.csv
9252 a29c15-05-2020_09_09_44_AM97001&7057.csv
9253 a29c15-05-2020_09_09_46_AM51003&1070.csv
9254 a29c15-05-2020_09_09_48_AM1852&917.csv
9255 a29c15-05-2020_09_09_49_AM7857&945.csv
9256 a29c15-05-2020_09_09_51_AM4889&979.csv
9257 a29c15-05-2020_09_09_52_AM1675&701.csv
9258 a29c15-05-2020_09_09_53_AM8755&847.csv
9259 a29c15-05-2020_09_09_55_AM5683&774.csv
9260 a29c15-05-2020_09_09_56_AM2577&661.csv
9261 a29c15-05-2020_09_09_57_AM9630&749.csv
9262 a29c15-05-2020_09_

9400 a29c15-05-2020_09_33_16_AM2542&666.csv
9401 a29c15-05-2020_09_33_17_AM5322&458.csv
error :  a29c15-05-2020_09_33_17_AM5322&458.csv
9402 a29c15-05-2020_09_33_18_AM8534&633.csv
9403 a29c15-05-2020_09_33_20_AM1583&714.csv
9404 a29c15-05-2020_09_33_21_AM4370&496.csv
error :  a29c15-05-2020_09_33_21_AM4370&496.csv
9405 a29c15-05-2020_09_33_22_AM7504&639.csv
9406 a29c15-05-2020_09_33_23_AM0611&759.csv
9407 a29c15-05-2020_09_33_25_AM4764&876.csv
9408 a29c15-05-2020_09_33_26_AM8711&801.csv
9409 a29c15-05-2020_09_33_28_AM2802&895.csv
9410 a29c15-05-2020_09_33_29_AM6490&522.csv
9411 a29c15-05-2020_09_33_30_AM0684&762.csv
9412 a29c15-05-2020_09_33_31_AM5591&693.csv
9413 a29c15-05-2020_09_33_33_AM9930&996.csv
9414 a29c15-05-2020_09_33_34_AM3764&809.csv
9415 a29c15-05-2020_09_33_36_AM7837&915.csv
9416 a29c15-05-2020_09_33_37_AM1731&797.csv
9417 a29c15-05-2020_09_33_38_AM5602&677.csv
9418 a29c15-05-2020_09_33_40_AM0679&766.csv
9419 a29c15-05-2020_09_33_41_AM6673&774.csv
9420 a29c15-05-2020_09_3

In [6]:
print(df.index[-1])
savefile()

8711
